# SHOULD PROBABLY SWAP OUT WATERSHED FOR SPOT FINDING OR KILL IT ENTIRELY

# Watershed segmentation

## Masking cells vs background

This is fine for finding the nucleii, but what if we wanted to find the whole cell, and keep them separate?  First we need to find a binary image of the cells.  The raw, unbackground subtracted lyso channel seems like a reasonable place to start.  First we'll blur it a little bit.

In [ ]:
smoothed_lyso = ndi.gaussian_filter(lyso, 10)
viewer.add_image(smoothed_lyso, name='smoothed_lyso', colormap='gray', blending='additive')

<Image layer 'smoothed_lyso' at 0x1b4b704e380>

If we adjust the contrast lower limit to around ~350 we can see the cells, so we'll use that as our threshold.

In [ ]:
masked_lyso = smoothed_lyso>350
viewer.add_image(masked_lyso, name='masked_lyso', colormap='gray', blending='additive')

<Image layer 'masked_lyso' at 0x1b4b6d0dba0>

## Separating the cells

Now we have a binary image of where there is cell vs no cell, but we have not separated them into individual cells.  To do so we are going to combine our nuclear mask with a little trick:  we are going to use the distance transform of the cell mask.  The distance transform is a measure of how far each pixel is from the nearest "edge" of the cell mask.  We can use this to "push" the cells apart from each other.

The distance transform is actually a pretty simple concept:  for each pixel measure how many pixels away a background (False) pixel is and that is your new intensity.

In [ ]:
edt = ndi.distance_transform_edt(masked_lyso)
viewer.add_image(edt, name='edt', colormap='gray', blending='additive')

<Image layer 'edt' at 0x1b4b6d0efe0>

ski.segmenation.watershed takes an image that can be used to decide when to break two objects apart (the edt), our seed image with the nucleii labeled (label_img), and a final mask that limits how far we can grow our cells (masked_lyso).

In [ ]:
watershedded = ski.segmentation.watershed(-edt, label_img, mask=masked_lyso)
viewer.add_labels(watershedded, name='watershedded', blending='additive')

<Labels layer 'watershedded' at 0x1b5000762f0>

If we turn the watershedded onto contour=1, and then adjust the contrast on the EDT, we can see kind of what it is doing.  It breaks the boundary between two cells where the distance transform is the smallest.

## Quantifying the cells

Watershedded is a label image, but that contains the whole cells now.  We can use the same regionprops_table function to quantify the cells.

In [ ]:
results = pd.DataFrame(ski.measure.regionprops_table(watershedded, lyso_backsub, properties=('label', 'area', 'centroid', 'mean_intensity')))
results

,label,area,centroid-0,centroid-1,mean_intensity
0,1,44235.0,188.152933,499.062462,71.550331
1,2,50295.0,131.969520,319.039407,54.592186
2,3,42868.0,243.536437,136.338947,73.122444
3,4,38548.0,397.226497,366.374883,94.249893


Let's say we want to also quantify the mitos, we will create a new table, but take the mito_intensity and just add it to the results table we already have by defining a new column 'mito_mean_intensity'

In [ ]:
lyso_results = pd.DataFrame(ski.measure.regionprops_table(watershedded, mito_backsub, properties=('label', 'area', 'centroid', 'mean_intensity')))
results['mito_mean_intensity'] = lyso_results['mean_intensity']
results

,label,area,centroid-0,centroid-1,mean_intensity,mito_mean_intensity
0,1,44235.0,188.152933,499.062462,71.550331,101.695312
1,2,50295.0,131.969520,319.039407,54.592186,108.867981
2,3,42868.0,243.536437,136.338947,73.122444,99.088104
3,4,38548.0,397.226497,366.374883,94.249893,106.858253
